In [5]:
import numpy as np
import pandas as pd
import seaborn as sns

In [6]:
datosML=pd.read_csv(r"D:\Bootcamp\MLproyecto\Data\Processed\datosML.csv",index_col=False)
datosML

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
0,2,0,4.0,1,1,1
1,3,0,2.0,2,2,2
2,3,0,4.0,2,2,1
3,4,0,2.0,2,2,4
4,4,0,1.0,3,1,1
...,...,...,...,...,...,...
1332,4,0,5.0,1,3,1
1333,4,0,5.0,9,1,1
1334,4,0,5.0,1,3,1
1335,4,0,5.0,3,3,2


In [7]:
datosML.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MES            1337 non-null   int64  
 1   Resultado      1337 non-null   int64  
 2   COMERCIAL_NUM  1337 non-null   float64
 3   TELEOP_NUM     1337 non-null   int64  
 4   CANAL_NUM      1337 non-null   int64  
 5   tipodeobra     1337 non-null   int64  
dtypes: float64(1), int64(5)
memory usage: 62.8 KB


In [8]:
datosML.describe()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
count,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000,1337.000000
mean,5.899776,0.225879,3.382947,3.305161,2.027674,3.359761
std,3.392993,0.418316,1.866810,2.782470,0.952454,1.606022
min,1.000000,0.000000,1.000000,1.000000,1.000000,1.000000
25%,3.000000,0.000000,2.000000,1.000000,1.000000,2.000000
50%,6.000000,0.000000,3.000000,2.000000,2.000000,4.000000
75%,9.000000,0.000000,5.000000,6.000000,3.000000,5.000000
max,12.000000,1.000000,7.000000,11.000000,4.000000,5.000000


In [9]:
datosML.corr()

,MES,Resultado,COMERCIAL_NUM,TELEOP_NUM,CANAL_NUM,tipodeobra
MES,1.000000,-0.016207,0.043288,-0.071522,0.087715,0.086977
Resultado,-0.016207,1.000000,0.091392,-0.018108,0.048173,-0.028575
COMERCIAL_NUM,0.043288,0.091392,1.000000,-0.029575,0.011716,-0.027511
TELEOP_NUM,-0.071522,-0.018108,-0.029575,1.000000,-0.706171,-0.118888
CANAL_NUM,0.087715,0.048173,0.011716,-0.706171,1.000000,0.080097
tipodeobra,0.086977,-0.028575,-0.027511,-0.118888,0.080097,1.000000


# Comprobación de equilibrio de target

In [49]:
datosML["Resultado"].value_counts()

0    1035
1     302
Name: Resultado, dtype: int64

In [11]:
#Dado que se se observa un desequilibrio en nuestro target, realizaremos un oversampling para solucionarlo

from imblearn.over_sampling import RandomOverSampler

oversampler = RandomOverSampler()
X_oversampled, y_oversampled = oversampler.fit_resample(datosML.drop("Resultado", axis=1), datosML["Resultado"])

# Creamos un DataFrame con el oversampling
datosoversampled = pd.DataFrame(X_oversampled, columns=datosML.drop("Resultado", axis=1).columns)
datosoversampled["Resultado"] = y_oversampled


In [12]:
y_oversampled.value_counts()

0    1035
1    1035
Name: Resultado, dtype: int64

# Pre modelos machine learning

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn import model_selection

In [14]:
X= X_oversampled
y= y_oversampled

In [15]:

validation_size = 0.20
seed = 7
X_train, X_test, y_train, y_test = model_selection.train_test_split(X,
                                                                    y,
                                                                    test_size=validation_size,
                                                                    random_state=seed)


## Fast check modelos

In [17]:
from lazypredict.Supervised import LazyClassifier
from sklearn.model_selection import train_test_split

clf= LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions= clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:02<00:00, 10.33it/s]


In [18]:
print(models)

                               Accuracy  Balanced Accuracy  ROC AUC  F1 Score  \
Model                                                                           
ExtraTreesClassifier               0.80               0.81     0.81      0.80   
RandomForestClassifier             0.79               0.80     0.80      0.79   
BaggingClassifier                  0.79               0.80     0.80      0.79   
ExtraTreeClassifier                0.78               0.78     0.78      0.78   
XGBClassifier                      0.78               0.78     0.78      0.78   
LabelPropagation                   0.78               0.78     0.78      0.77   
DecisionTreeClassifier             0.78               0.78     0.78      0.77   
LabelSpreading                     0.77               0.78     0.78      0.77   
LGBMClassifier                     0.73               0.74     0.74      0.73   
NuSVC                              0.73               0.73     0.73      0.73   
KNeighborsClassifier        

## MODELO 1: LOGISTIC REGRESION

In [19]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [20]:
prediction= model.predict(X_test)


In [21]:
accuracyregression=  model.score(X_test, y_test)
accuracyregression

0.5193236714975845

In [22]:
name='Logistic Regression'
kfold = model_selection.KFold(n_splits=10) #Parte los datos en 10 trozos para usar validación cruzada / cross validation
cv_results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')

msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
print(cv_results)

[0.53614458 0.56024096 0.54216867 0.53012048 0.54216867 0.53012048
 0.51515152 0.55757576 0.54545455 0.46060606]


## MODELO 2: RANDOM TREE

In [23]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

dtr = DecisionTreeRegressor()
dtr.fit(X_train,y_train)

DecisionTreeRegressor()

In [24]:
dtrprediction= dtr.predict(X_test)

In [25]:
dtraccuracy= dtr.score (X_test, y_test)
dtraccuracy

0.28668937836160335

In [26]:
dtraccscores = cross_val_score(dtr, X, y, cv=5)
dtraccscores

array([-1.96165626, -1.07517671, -0.91693161, -0.25988234,  0.        ])

## MODELO 3: BAGGING

In [27]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier


dtc = DecisionTreeClassifier()
modelbag = BaggingClassifier(base_estimator=dtc, n_estimators=100, random_state=seed)

In [28]:
modelbag.fit(X_train, y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=100,
                  random_state=7)

In [29]:
bagpred = modelbag.predict(X_test)
bagcuracy=modelbag.score(X_test, y_test)
bagcuracy

0.7971014492753623

In [30]:
bagscores = cross_val_score(modelbag, X, y, cv=5)
bagscores

array([0.69082126, 0.6352657 , 0.74154589, 0.70289855, 0.71014493])

In [31]:
mean_score = bagscores.mean()
print("Mean Accuracy:", mean_score)

Mean Accuracy: 0.6961352657004831


In [32]:
# pruebas extra para validar la fiabilidad

from sklearn.metrics import precision_score, recall_score, f1_score

In [33]:
#precision 
precision = precision_score(y_test, bagpred)
precision

0.7375

In [34]:
#recall

recall = recall_score(y_test, bagpred)
recall

0.8939393939393939

In [35]:
#f1 score
f1 = f1_score(y_test, bagpred)
f1

0.8082191780821918

## MODELO 4: Ada Boost

In [36]:
from sklearn.ensemble import AdaBoostClassifier

adaboost = AdaBoostClassifier()
adaboost.fit(X_train, y_train)


AdaBoostClassifier()

In [37]:
adapred=adaboost.predict(X_test)


In [38]:
adaaccuracy=adaboost.score(X_test, y_test)
adaaccuracy

0.6183574879227053

In [39]:
adaboostscores = cross_val_score(adaboost, X, y, cv=5)
adaboostscores

array([0.34057971, 0.37198068, 0.44927536, 0.48067633, 0.53140097])

## MODELO 5: RANDOM FOREST

In [40]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)


In [41]:
rf_classifier.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [42]:
y_pred = rf_classifier.predict(X_test)


In [43]:
accuracy_randomforest = accuracy_score(y_test, y_pred)
accuracy_randomforest

0.7995169082125604

In [44]:
#Cross validation
forestscores = cross_val_score(rf_classifier, X, y, cv=5)
forestscores

array([0.65217391, 0.66425121, 0.74154589, 0.73913043, 0.74637681])

In [45]:
mean_forest=forestscores.mean()
mean_forest

0.7086956521739131

## Modelo 6: Extra Trees Classifier 

In [52]:
from sklearn.ensemble import ExtraTreesClassifier

clf = ExtraTreesClassifier()

# Entrenar el modelo con tus datos de entrenamiento
clf.fit(X_train, y_train)

# Realizar predicciones en tus datos de prueba
y_pred = clf.predict(X_test)



In [54]:
# Calcular la precisión del modelo
accuracy = clf.score(X_test, y_test)
accuracy

0.8043478260869565

In [57]:
#cross validation

extratreecross=cross_val_score(clf, X, y, cv=5)
extratreecross.mean()

0.7328502415458937

## MEJOR MODELO: Extra Trees Classifier 

In [58]:
import joblib

joblib.dump(clf, 'new_model.pkl')

['new_model.pkl']

# FEATURE IMPORTANCE

In [59]:
#feature importance


# Obtener la importancia de las características
importances = clf.feature_importances_

# Crear un DataFrame con las características y su importancia
feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})

# Ordenar las características por su importancia descendente
feature_importances = feature_importances.sort_values('Importance', ascending=False)

# Imprimir el ranking de importancia de las características
print(feature_importances)


         Feature  Importance
0            MES        0.40
1  COMERCIAL_NUM        0.22
2     TELEOP_NUM        0.18
4     tipodeobra        0.15
3      CANAL_NUM        0.05
